In [1]:
import os
from openpyxl import load_workbook
import pandas as pd

in_path = "농축수산물 품목 및 등급 코드표.xlsx"
out_path = "./output/부류코드.json"

def find_header_row(ws):
    """워크시트에서 '부류코드', '부류명' 컬럼이 포함된 헤더 행을 찾아 반환"""
    for i, row in enumerate(ws.iter_rows(min_row=1, max_row=50, values_only=True)):
        row_norm = [str(x).strip() if isinstance(x, str) else x for x in row]
        if ("부류코드" in row_norm) and ("부류명" in row_norm):
            return i + 1, row_norm
    raise RuntimeError("헤더 행(부류코드/부류명)을 찾지 못했습니다.")

def get_column_indices(header_cells):
    """헤더 셀에서 '부류코드', '부류명' 컬럼의 인덱스를 계산하여 반환"""
    col_idx_code = header_cells.index("부류코드") + 1
    col_idx_name = header_cells.index("부류명") + 1
    return col_idx_code, col_idx_name

def extract_data(ws, header_row_idx, col_idx_code, col_idx_name):
    """워크시트에서 지정된 컬럼의 데이터를 추출하여 딕셔너리 리스트로 반환"""
    data = []
    for row in ws.iter_rows(min_row=header_row_idx+1, 
                            min_col=min(col_idx_code, col_idx_name),
                            max_col=max(col_idx_code, col_idx_name), 
                            values_only=True):
        code = row[(col_idx_code - min(col_idx_code, col_idx_name))]
        name = row[(col_idx_name - min(col_idx_code, col_idx_name))]
        
        if isinstance(code, str): 
            code = code.strip()
        if isinstance(name, str): 
            name = name.strip()
        
        data.append({"부류코드": code, "부류명": name})
    return data

def clean_and_save(data, output_path):
    """데이터를 정리하고 JSON 파일로 저장"""
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    df = pd.DataFrame(data, columns=["부류코드", "부류명"])
    df.to_json(output_path, orient="records", indent=4, force_ascii=False)

def main():
    """메인 실행 함수"""
    wb = load_workbook(in_path, data_only=True, read_only=True)
    ws = wb["부류코드"]

    header_row_idx, header_cells = find_header_row(ws)
    col_idx_code, col_idx_name = get_column_indices(header_cells)
    data = extract_data(ws, header_row_idx, col_idx_code, col_idx_name)
    clean_and_save(data, out_path)

if __name__ == "__main__":
    main()

In [2]:
import os
from openpyxl import load_workbook
import pandas as pd

in_path = "농축수산물 품목 및 등급 코드표.xlsx"
out_path = "./output/품목코드.json"

def find_header_row(ws):
    """워크시트에서 '부류코드', '품목코드', '품목명' 컬럼이 포함된 헤더 행을 찾아 반환"""
    for i, row in enumerate(ws.iter_rows(min_row=1, max_row=50, values_only=True), start=1):
        r = [x.strip() if isinstance(x, str) else x for x in row]
        if "부류코드" in r and "품목코드" in r and "품목명" in r:
            return i, r
    raise AssertionError("헤더(부류코드/품목코드/품목명) 행을 찾지 못했습니다.")

def get_column_indices(headers):
    """헤더에서 필요한 컬럼들의 인덱스를 계산하여 반환"""
    return {name: headers.index(name) + 1 for name in ["부류코드", "품목코드", "품목명"]}

def extract_data(ws, header_row, idx):
    """워크시트에서 지정된 컬럼의 데이터를 추출하여 딕셔너리 리스트로 반환"""
    data = []
    for row in ws.iter_rows(min_row=header_row+1,
                            min_col=min(idx.values()), max_col=max(idx.values()),
                            values_only=True):
        rec = {k: row[idx[k]-min(idx.values())] for k in idx}
        
        if all((v is None or (isinstance(v, str) and v.strip() == "")) for v in rec.values()):
            continue
            
        for k, v in rec.items():
            if isinstance(v, str): 
                rec[k] = v.strip()
        data.append(rec)
    return data

def clean_and_save(data, output_path):
    """데이터를 정리하고 JSON 파일로 저장"""
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    df = pd.DataFrame(data, columns=["부류코드","품목코드","품목명"])
    df.to_json(output_path, orient="records", indent=4, force_ascii=False)

def main():
    """메인 실행 함수"""
    wb = load_workbook(in_path, data_only=True, read_only=True)
    ws = wb["품목코드"]

    header_row, headers = find_header_row(ws)
    idx = get_column_indices(headers)
    data = extract_data(ws, header_row, idx)
    clean_and_save(data, out_path)

if __name__ == "__main__":
    main()

In [3]:
import os
from openpyxl import load_workbook
import pandas as pd

in_path = "농축수산물 품목 및 등급 코드표.xlsx"
out_path = "./output/품종코드.json"
wanted = ["품목 코드", "품종코드", "품목명", "품종명", 
          "도매출하단위", "도매출하단위 크기", "소매출하단위", "소매출하단위 크기",
             "친환경농산물출하단위('20.4~)", "친환경농산물출하단위크기('20.4~)",
               "도매 등급", "소매 등급", "친환경 등급('20.4~)"]

def find_header_row(ws, required_columns):
    """워크시트에서 필요한 컬럼들이 모두 포함된 헤더 행을 찾아 반환"""
    for i, row in enumerate(ws.iter_rows(min_row=1, max_row=50, values_only=True), start=1):
        r = [x.strip() if isinstance(x, str) else x for x in row]
        if all(w in r for w in required_columns):
            return i, r
    raise AssertionError(f"헤더({'/'.join(required_columns)}) 행을 찾지 못했습니다.")

def get_column_indices(headers, required_columns):
    """헤더에서 필요한 컬럼들의 인덱스를 계산하여 반환"""
    return {name: headers.index(name) + 1 for name in required_columns}

def extract_data(ws, header_row, idx):
    """워크시트에서 지정된 컬럼의 데이터를 추출하여 딕셔너리 리스트로 반환"""
    data = []
    for row in ws.iter_rows(min_row=header_row+1,
                            min_col=min(idx.values()), max_col=max(idx.values()),
                            values_only=True):
        rec = {k: row[idx[k]-min(idx.values())] for k in idx}
        
        if all((v is None or (isinstance(v, str) and v.strip() == "")) for v in rec.values()):
            continue
            
        for k, v in rec.items():
            if isinstance(v, str): 
                rec[k] = v.strip()
        data.append(rec)
    return data

def clean_and_save(data, output_path, columns):
    """데이터를 정리하고 JSON 파일로 저장"""
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    df = pd.DataFrame(data, columns=columns)
    df.to_json(output_path, orient="records", indent=4, force_ascii=False)

def main():
    """메인 실행 함수"""
    wb = load_workbook(in_path, data_only=True, read_only=True)
    ws = wb["품종코드"]

    header_row, headers = find_header_row(ws, wanted)
    idx = get_column_indices(headers, wanted)
    data = extract_data(ws, header_row, idx)
    clean_and_save(data, out_path, wanted)

if __name__ == "__main__":
    main()

In [4]:
import os
from openpyxl import load_workbook
import pandas as pd

in_path = "농축수산물 품목 및 등급 코드표.xlsx"
out_path = "./output/코드통합.json"
wanted = ["품목 그룹코드", "품목 그룹명", "품목 코드", "품목명", "품종코드", "품종명", 
          "도매출하단위", "도매출하단위 크기", "소매출하단위", "소매출하단위 크기",
             "친환경농산물출하단위('20.4~)", "친환경농산물출하단위크기('20.4~)",
               "도매 등급", "소매 등급", "친환경 등급('20.4~)"]

rename_map = {
    "품목 그룹코드": "부류코드",
    "품목 그룹명": "부류명"
}

def find_header_row(ws, required_columns):
    for i, row in enumerate(ws.iter_rows(min_row=1, max_row=50, values_only=True), start=1):
        r = [x.strip() if isinstance(x, str) else x for x in row]
        if all(w in r for w in required_columns):
            return i, r
    raise AssertionError(f"헤더({'/'.join(required_columns)}) 행을 찾지 못했습니다.")

def get_column_indices(headers, required_columns):
    return {name: headers.index(name) + 1 for name in required_columns}

def extract_data(ws, header_row, idx):
    data = []
    for row in ws.iter_rows(min_row=header_row+1,
                            min_col=min(idx.values()), max_col=max(idx.values()),
                            values_only=True):
        rec = {k: row[idx[k]-min(idx.values())] for k in idx}
        
        if all((v is None or (isinstance(v, str) and v.strip() == "")) for v in rec.values()):
            continue
            
        for k, v in rec.items():
            if isinstance(v, str): 
                rec[k] = v.strip()
        data.append(rec)
    return data

def clean_and_save(data, output_path, columns, rename_map):
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    df = pd.DataFrame(data, columns=columns)
    df = df.rename(columns=rename_map)
    df.to_json(output_path, orient="records", indent=4, force_ascii=False)

def main():
    wb = load_workbook(in_path, data_only=True, read_only=True)
    ws = wb["코드통합(부류+품목+품종코드)"]

    header_row, headers = find_header_row(ws, wanted)
    idx = get_column_indices(headers, wanted)
    data = extract_data(ws, header_row, idx)
    clean_and_save(data, out_path, wanted, rename_map)

if __name__ == "__main__":
    main()

In [5]:
import os
from openpyxl import load_workbook
import pandas as pd

in_path = "농축수산물 품목 및 등급 코드표.xlsx"
out_path = "./output/등급코드.json"
wanted = ["등급코드(p_productrankcode)", "등급코드(p_graderank)", "등급코드명"]

def find_header_row(ws, required_columns):
    """워크시트에서 필요한 컬럼들이 모두 포함된 헤더 행을 찾아 반환"""
    for i, row in enumerate(ws.iter_rows(min_row=1, max_row=50, values_only=True), start=1):
        r = [x.strip() if isinstance(x, str) else x for x in row]
        if all(w in r for w in required_columns):
            return i, r
    raise AssertionError(f"헤더({'/'.join(required_columns)}) 행을 찾지 못했습니다.")

def get_column_indices(headers, required_columns):
    """헤더에서 필요한 컬럼들의 인덱스를 계산하여 반환"""
    return {name: headers.index(name) + 1 for name in required_columns}

def extract_data(ws, header_row, idx):
    """워크시트에서 지정된 컬럼의 데이터를 추출하여 딕셔너리 리스트로 반환"""
    data = []
    for row in ws.iter_rows(min_row=header_row+1,
                            min_col=min(idx.values()), max_col=max(idx.values()),
                            values_only=True):
        rec = {k: row[idx[k]-min(idx.values())] for k in idx}
        
        if all((v is None or (isinstance(v, str) and v.strip() == "")) for v in rec.values()):
            continue
            
        for k, v in rec.items():
            if isinstance(v, str): 
                rec[k] = v.strip()
        data.append(rec)
    return data

def clean_and_save(data, output_path, columns):
    """데이터를 정리하고 JSON 파일로 저장"""
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    df = pd.DataFrame(data, columns=columns)
    df.to_json(output_path, orient="records", indent=4, force_ascii=False)
    print(f"Saved JSON: {output_path}, rows={len(data)}")

def main():
    """메인 실행 함수"""
    wb = load_workbook(in_path, data_only=True, read_only=True)
    ws = wb["등급코드"]

    header_row, headers = find_header_row(ws, wanted)
    idx = get_column_indices(headers, wanted)
    data = extract_data(ws, header_row, idx)
    clean_and_save(data, out_path, wanted)

if __name__ == "__main__":
    main()

Saved JSON: ./output/등급코드.json, rows=29


In [6]:
import os
from openpyxl import load_workbook
import pandas as pd

in_path = "농축수산물 품목 및 등급 코드표.xlsx"
out_path = "./output/축산물코드.json"
wanted = ["품목명", "품종명", "등급명", "품목코드(itemcode)", "품종코드(kindcode)", "등급코드(periodProductList)"]

rename_map = {
    "품목코드(itemcode)": "품목코드",
    "품종코드(kindcode)": "품종코드",
}

def find_header_row(ws, required_columns):
    """워크시트에서 필요한 컬럼들이 모두 포함된 헤더 행을 찾아 반환"""
    for i, row in enumerate(ws.iter_rows(min_row=1, max_row=50, values_only=True), start=1):
        r = [x.strip() if isinstance(x, str) else x for x in row]
        if all(w in r for w in required_columns):
            return i, r
    raise AssertionError(f"헤더({'/'.join(required_columns)}) 행을 찾지 못했습니다.")

def get_column_indices(headers, required_columns):
    """헤더에서 필요한 컬럼들의 인덱스를 계산하여 반환"""
    return {name: headers.index(name) + 1 for name in required_columns}

def extract_data(ws, header_row, idx):
    """워크시트에서 지정된 컬럼의 데이터를 추출하여 딕셔너리 리스트로 반환"""
    data = []
    for row in ws.iter_rows(min_row=header_row+1,
                            min_col=min(idx.values()), max_col=max(idx.values()),
                            values_only=True):
        rec = {k: row[idx[k]-min(idx.values())] for k in idx}
        
        if all((v is None or (isinstance(v, str) and v.strip() == "")) for v in rec.values()):
            continue
            
        for k, v in rec.items():
            if isinstance(v, str): 
                rec[k] = v.strip()
        data.append(rec)
    return data

def clean_and_save(data, output_path, columns):
    """데이터를 정리하고 JSON 파일로 저장"""
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    df = pd.DataFrame(data, columns=columns)
    df = df.rename(columns=rename_map)
    df.to_json(output_path, orient="records", indent=4, force_ascii=False)
    print(f"Saved JSON: {output_path}, rows={len(data)}")

def main():
    """메인 실행 함수"""
    wb = load_workbook(in_path, data_only=True, read_only=True)
    ws = wb["축산물 코드"]

    header_row, headers = find_header_row(ws, wanted)
    idx = get_column_indices(headers, wanted)
    data = extract_data(ws, header_row, idx)
    clean_and_save(data, out_path, wanted)

if __name__ == "__main__":
    main()

Saved JSON: ./output/축산물코드.json, rows=56


In [7]:
import os
from openpyxl import load_workbook
import pandas as pd

in_path = "농축수산물 품목 및 등급 코드표.xlsx"
out_path = "./output/산물코드.json"
wanted = ["산물번호","산물분류명","산물분류코드","품목분류명","품목분류코드",
          "품목명","품목코드","품종명","품종코드","산물등급명","산물등급코드","산물부류별_단위"]

rename_map = {
    "품목분류코드": "부류코드",
    "품목분류명": "부류명",
    "산물등급명": "등급코드명",
    "산물등급코드": "등급코드(p_productrankcode)"
}

def find_header_row(ws, required_columns):
    """워크시트에서 필요한 컬럼들이 모두 포함된 헤더 행을 찾아 반환"""
    for i, row in enumerate(ws.iter_rows(min_row=1, max_row=50, values_only=True), start=1):
        r = [x.strip() if isinstance(x, str) else x for x in row]
        if all(w in r for w in required_columns):
            return i, r
    raise AssertionError(f"헤더({'/'.join(required_columns)}) 행을 찾지 못했습니다.")

def get_column_indices(headers, required_columns):
    """헤더에서 필요한 컬럼들의 인덱스를 계산하여 반환"""
    return {name: headers.index(name) + 1 for name in required_columns}

def extract_data(ws, header_row, idx):
    """워크시트에서 지정된 컬럼의 데이터를 추출하여 딕셔너리 리스트로 반환"""
    data = []
    for row in ws.iter_rows(min_row=header_row+1,
                            min_col=min(idx.values()), max_col=max(idx.values()),
                            values_only=True):
        rec = {k: row[idx[k]-min(idx.values())] for k in idx}
        
        if all((v is None or (isinstance(v, str) and v.strip() == "")) for v in rec.values()):
            continue
            
        for k, v in rec.items():
            if isinstance(v, str): 
                rec[k] = v.strip()
        data.append(rec)
    return data

def clean_and_save(data, output_path, columns):
    """데이터를 정리하고 JSON 파일로 저장"""
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    df = pd.DataFrame(data, columns=columns)
    df = df.rename(columns=rename_map)
    df.to_json(output_path, orient="records", indent=4, force_ascii=False)
    print(f"Saved JSON: {output_path}, rows={len(data)}")

def main():
    """메인 실행 함수"""
    wb = load_workbook(in_path, data_only=True, read_only=True)
    ws = wb["산물코드"]

    header_row, headers = find_header_row(ws, wanted)
    idx = get_column_indices(headers, wanted)
    data = extract_data(ws, header_row, idx)
    clean_and_save(data, out_path, wanted)

if __name__ == "__main__":
    main()

Saved JSON: ./output/산물코드.json, rows=718
